## Setup

In [1]:
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)

Versión de TensorFlow: 2.4.0


If you haven't installed the following dependencies, run:

In [2]:
!sudo apt-get install -y xvfb ffmpeg
!pip install gym
!pip install 'imageio==2.4.0'
!pip install PILLOW
!pip install pyglet
!pip install pyvirtualdisplay
!pip install tf-agents
!pip install tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 784 kB in 1s (650 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pack

In [3]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [4]:
tf.compat.v1.enable_v2_behavior()



In [5]:
tf.version.VERSION

'2.4.0'

In [6]:
!pip install pandas-datareader


In [7]:
!pip install pip yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 1.5MB 7.5MB/s 
     |████████████████████████████████| 5.5MB 24.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=2528f8f29e60c3fdc173e9322f5b09c926768909850c67a37922df3546b46986
  Stored in directory: /tmp/pip-ephem-wheel-cache-vx2ksb1x/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


## Hyperparameters

In [8]:
num_iterations = 200000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 2  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-4  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## datas data

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import beta
from sklearn import preprocessing
from pandas_datareader import data as pdr
#import yfinance as yf
import fix_yahoo_finance as yf
yf.pdr_override()
import os 
from datetime import date

In [10]:
start="2020-09-01"
end="2020-12-31"
#L=2500#periodo de los ultimos dias a analizar (maximo 250)
M=0 #ultimo periodo que excluimos del registro para evitar el leakage
la=32 #dias anteriores utilizados para calcular media y varianza del dia presente
co=int(la/4)


CO="CO"

LA="LA" #string acorde con a

In [11]:
class Betas():
  def __init__(self,ticker,la=34,co=int(34/4),start="2010-01-01",end="2020-12-31"):
    self.ticker=ticker
    self.la=la
    self.co=co
    self.start=start
    self.end=end
    self.list=["Open","High","Low","Close"] 
    #self.list=["Close"]
  
  def betas(self):
    from pandas_datareader import data as pdr
    from scipy.stats import beta
    datau=pdr.get_data_yahoo(self.ticker,self.start,self.end,progress=False)
    datau['LogReturn'] = np.log(datau['Close']).diff() 
    datau["P"+"LA"]=0
    datau["N"+"LA"]=0
    datau["P"+"CO"]=0
    datau["N"+"CO"]=0

    for i in self.list:
      datau[i+"R"]=(datau[i]-datau[i].shift(1))*100/datau[i].shift(1)
      datau[i+"R"]=datau[i+"R"].dropna()
      datau[i+"P"] = datau[i+"R"].where(datau[i+"R"] > 0.0, 1e-6)
      datau[i+"N"] = -datau[i+"R"].where(datau[i+"R"] <= 0.0, 1e-6)
  
      datau[i+"P"+"LA"]=datau[i+"P"].cumsum()-datau[i+"P"].shift(self.la).cumsum()+1.0
      datau[i+"N"+"LA"]=datau[i+"N"].cumsum()-datau[i+"N"].shift(self.la).cumsum()+1.0
      datau[i+"P"+"CO"]=datau[i+"P"].cumsum()-datau[i+"P"].shift(self.co).cumsum()+0.25
      datau[i+"N"+"CO"]=datau[i+"N"].cumsum()-datau[i+"N"].shift(self.co).cumsum()+0.25

      datau["P"+"LA"]=datau["P"+"LA"]+datau[i+"P"+"LA"]
      datau["N"+"LA"]=datau["N"+"LA"]+datau[i+"N"+"LA"]
      datau["P"+"CO"]=datau["P"+"CO"]+datau[i+"P"+"CO"]
      datau["N"+"CO"]=datau["N"+"CO"]+datau[i+"N"+"CO"]

    
    datau=datau.dropna()    

    datau["mean"+"LA"]= beta.stats(datau["P"+"LA"], datau["N"+"LA"], moments='m')
    datau["mean"+"CO"]= beta.stats(datau["P"+"CO"], datau["N"+"CO"], moments='m')

    

    datau = datau.reset_index()    

    
    datau["meanCO-8"]=datau["meanCO"].shift(8)
    datau["meanCO-16"]=datau["meanCO"].shift(16)
    datau["meanCO-24"]=datau["meanCO"].shift(24)




    datau=datau[["Date","Open","Close","meanLA","meanCO","LogReturn","meanCO-8","meanCO-16","meanCO-24"]]
    datau=datau.dropna()
    return datau

In [12]:
#CARTERA REVISADA

igbm=["A3M.MC","ABG.MC","ACS.MC","ACX.MC","ADX.MC","ADZ.MC","AEDAS.MC","AENA.MC","AI.MC","AIR.MC","ALB.MC","ALM.MC","ALNT.MC","AMP.MC","AMS.MC","ANA.MC","APAM.MC","APPS.MC","ARM.MC",
      "AZK.MC","BAIN.MC","BBVA.MC","BDL.MC","BIO.MC","BKIA.MC","BKT.MC","BKY.MC","BME.MC","CABK.MC","CAF.MC","CASH.MC","CBAV.MC","CCEP.MC","CDR.MC","CEV.MC","CIE.MC","CLEO.MC","CLNX.MC",
      "CMC.MC","COL.MC","DESA.MC","DIA.MC","DOM.MC","EAT.MC","EBRO.MC","ECR.MC","EDR.MC","EKT.MC","ELE.MC","ENC.MC","ENG.MC","ENO.MC","EZE.MC","FAE.MC","FCC.MC","FDR.MC","FER.MC","GALQ.MC",
      "GCO.MC","GEST.MC","GRE.MC","GRF.MC","GSJ.MC","HOME.MC","IAG.MC","IBE.MC","IBG.MC","IDR.MC","ISUR.MC","ITX.MC","LBK.MC","LGT.MC","LOG.MC","LRE.MC","MAP.MC","MAS.MC","MCM.MC","MDF.MC",
      "MEL.MC","MRL.MC","MTB.MC","MTS.MC","MVC.MC","NEA.MC","NHH.MC","NTGY.MC","NTH.MC","NXT.MC","NYE.MC","OHL.MC","OLE.MC","ORY.MC","PHM.MC","PRM.MC",
      "PRS.MC","PSG.MC","PVA.MC","QBT.MC","R4.MC","RDM.MC","REE.MC","REN.MC","REP.MC","RIO.MC","RJF.MC","RLIA.MC","ROVI.MC","SAB.MC","SAN.MC","SCYR.MC","SGRE.MC",
      "SLR.MC","SPK.MC","SPS.MC","TEF.MC","TL5.MC","TLGO.MC","TRE.MC","TRG.MC","TUB.MC","UBS.MC","UNI.MC","VER.MC","VID.MC","VIS.MC","VOC.MC","ZOT.MC"]
cac=["AC.PA","AI.PA","AIR.PA","MT.AS","ATO.PA","CS.PA","BNP.PA","CAP.PA","CA.PA","BN.PA","DSY.PA","ENGI.PA","EL.PA","RMS.PA","KER.PA","OR.PA","LR.PA","MC.PA","ML.PA","ORA.PA","RI.PA","UG.PA","PUB.PA","RNO.PA","SAF.PA","SGO.PA","SAN.PA","SU.PA","GLE.PA","SW.PA","STM.PA","HO.PA","FP.PA","VIE.PA","DG.PA","VIV.PA","WLN.PA"]
dow=["MCD","PG","JPM","AAPL","JPM","GOOG","GS","TRV","AXP","PFE","BA","JNJ","UNH","VZ","MRK","NKE","CSCO","IBM","CAT","DIS","KO","WBA","MSFT","MMM","INTC","WMT","XOM","CVX","HD","ALXN","BIDU"]
ibex=["ANA.MC","ACX.MC","ACS.MC","AENA.MC","ALM.MC","AMS.MC","MTS.MC" ,"SAB.MC","SAN.MC","BKIA.MC","BKT.MC","BBVA.MC","CABK.MC","CLNX.MC","CIE.MC","ENG.MC","ELE.MC","FER.MC","GRF.MC","IAG.MC","IBE.MC","ITX.MC","IDR.MC","COL.MC","MAP.MC","MEL.MC","MRL.MC","NTGY.MC","PHM.MC","REE.MC","REP.MC","SGRE.MC","SLR.MC","TEF.MC","VIS.MC"]
#ibex=["BBVA.MC","TL5.MC","TEF.MC","MRL.MC","NTGY.MC","REE.MC","REP.MC","SAN.MC","TEF.MC","VIS.MC","ANA.MC","ACX.MC","ACS.MC","AENA.MC","MTS.MC","BKT.MC","BBVA.MC","CABK.MC","CLNX.MC","COL.MC","ELE.MC","FER.MC","IAG.MC","IBE.MC","ITX.MC","IDR.MC","MAP.MC","GRF.MC","MEL.MC","SGRE.MC"]
dax=["BEI.DE","FRE.DE","LHA.DE","SIE.DE","DAI.DE","VNA.DE","WDI.DE","DPW.DE","RWE.DE","HEI.DE","MRK.DE","SAP.DE","ALV.DE","IFX.DE","FME.DE","DB1.DE","MUV2.DE","EOAN.DE","CON.DE","BAYN.DE","DBK.DE","VOW3.DE","ADS.DE","BMW.DE","DTE.DE","HEN3.DE"]
nasdaq=["ATVI","ADBE","AMD","ALXN","ALGN","GOOG","GOOGL","AMZN","AMGN","ADI","ANSS","AAPL","AMAT","ASML","ADSK","ADP","BIDU","BIIB","BMRN","BKNG","AVGO","CDNS","CDW","CERN","CHTR","CHKP","CTAS","CSCO","CTXS","CTSH","CMCSA","CPRT",
        "CSGP","COST","CSX","DXCM","DLTR","EBAY","EA","EXC","EXPE","FB","FAST","FISV","GILD","IDXX","ILMN","INCY","INTC","INTU","ISRG","JD","KLAC","LRCX","LBTYA","LBTYK","LULU","MAR","MXIM","MELI","MCHP","MU","MSFT",
        "MDLZ","MNST","NTAP","NTES","NFLX","NVDA","NXPI","ORLY","PCAR","PAYX","PYPL","PEP","QCOM","REGN","ROST","SGEN","SIRI","SWKS","SPLK","SBUX","SNPS","TMUS","TTWO","TSLA","TXN","KHC","TCOM","ULTA","UAL","VRSN","VRSK","VRTX",
        "WBA","WDC","WDAY","XEL","XLNX"]
s_and_p_100=["AA","AAPL","ABT","AEP","ALL","AMZN","AXP","BA","BAC","BAX","BK","BMY","C","CAT","CL","CMCSA","COF","COP","COST","CPB","CSCO","CVS","CVX","DIS","DVN","ETR","EXC","F","FCX","FDX","GD","GE",
             "GILD","GOOG","GS","HAL","HD","HON","HPQ","IBM","JNJ","JPM","KO","LMT","LOW","MCD","MDT","MET","MMM","MO","MON","MRK","MS","MSFT","NKE","NOV","NSC","NWSA","ORCL","OXY","PEP","PFE","PG","PM","QCOM","RF",
             "SLB","SO","T","TGT","TWX","TXN","UNH","UPS","USB","VZ","WFC","WMB","WMT","WY","XOM","XRX"]

mib=["SPM.MI","DIA.MI","FBK.MI","FCA.MI","BAMI.MI","PRY.MI","AZM.MI","TIT.MI","UNI.MI","BMED.MI","BZU.MI","RACE.MI","LDO.MI","MB.MI","PIRC.MI","ATL.MI","ENI.MI","G.MI","PST.MI","CPR.MI","UBI.MI","TEN.MI","REC.MI","IG.MI","TRN.MI","SRG.MI","IP.MI","CNHI.MI","MONC.MI","STM.MI"]

total=ibex+cac+dax+mib+dow+nasdaq+s_and_p_100
europa=ibex+cac+dax+mib
america=dow+nasdaq+s_and_p_100
america2=nasdaq+s_and_p_100
total2=ibex+cac+dax+mib+s_and_p_100+nasdaq
mercados=[igbm,ibex,cac,dax,mib,dow,nasdaq,s_and_p_100]

len(total)

341

In [13]:
mert=pd.DataFrame()

for ticker in ibex:
  mer=Betas(ticker,start="2020-10-10",end="2021-12-31").betas()
  mer["ticker"]=ticker
  mert = pd.concat([mert, mer], ignore_index=True, sort=False)


In [14]:
data=mert.copy()
L=len(data)

data.head(12)

,Date,Open,Close,meanLA,meanCO,LogReturn,meanCO-8,meanCO-16,meanCO-24,ticker
0,2021-01-04,116.900002,117.099998,0.663225,0.789782,0.003422,0.475895,0.755104,0.746101,ANA.MC
1,2021-01-05,117.300003,116.500000,0.652152,0.677627,-0.005137,0.526091,0.745408,0.800470,ANA.MC
2,2021-01-06,117.300003,119.500000,0.669318,0.631131,0.025425,0.693920,0.583734,0.733978,ANA.MC
3,2021-01-07,120.000000,126.099998,0.693347,0.787707,0.053759,0.668118,0.694615,0.588785,ANA.MC
4,2021-01-08,127.699997,127.900002,0.707187,0.837368,0.014173,0.689448,0.672709,0.632743,ANA.MC
5,2021-01-11,127.099998,124.599998,0.682657,0.745728,-0.026140,0.658039,0.694207,0.707450,ANA.MC
6,2021-01-12,125.699997,125.300003,0.684349,0.769990,0.005602,0.567071,0.724846,0.682663,ANA.MC
7,2021-01-13,122.900002,126.699997,0.681642,0.736151,0.011111,0.562351,0.670849,0.692718,ANA.MC
8,2021-01-14,127.000000,126.900002,0.679634,0.766955,0.001577,0.789782,0.475895,0.755104,ANA.MC
9,2021-01-15,126.599998,125.400002,0.663248,0.739913,-0.011891,0.677627,0.526091,0.745408,ANA.MC


In [15]:
#plt.scatter(data["Tendencia"], data["LogReturn"])

## Betas environment

In [16]:


import abc
import six

from tf_agents.trajectories import time_step as ts
from tf_agents.utils import common
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

In [17]:
class PyEnvironment(object):

  def reset(self):
    """Return initial_time_step."""
    self._current_time_step = self._reset()
    return self._current_time_step

  def step(self, action):
    """Apply action and return new time_step."""
    if self._current_time_step is None:
        return self.reset()
    self._current_time_step = self._step(action)
    return self._current_time_step

  def current_time_step(self):
    return self._current_time_step

  def time_step_spec(self):
    """Return time_step_spec."""

  @abc.abstractmethod
  def observation_spec(self):
    """Return observation_spec."""

  @abc.abstractmethod
  def action_spec(self):
    """Return action_spec."""

  @abc.abstractmethod
  def _reset(self):
    """Return initial_time_step."""

  @abc.abstractmethod
  def _step(self, action):
    """Apply action and return new time_step."""
    self._current_time_step = self._step(action)
    return self._current_time_step

In [18]:
# ATENCION: Todas las variables deben ir precedidas con self._

class BetamEnv(py_environment.PyEnvironment):

  def __init__(self,data):
    self._C=5000
    #self._ticker_comprado=self._data.loc[0,"ticker"]  #atributo para evitar que se realicen operaciones de venta con un accion diferente a la que se haya comprado
    self._IA=0       #inversion acumulada
    self._VA=0       #valor actual de la accion
    self._VC=0        #valor de compra de la accion
    self._na=0        # numero acciones compradas
    self._i=3
    self._reward=0
    self._data=data
    self._invested=0
    self._Lon=len(self._data)-2
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,7), dtype=np.float32, minimum=0, maximum=1,  name='observation')
    self._state=self._data.loc[0,["meanLA","meanCO","meanLA-4","meanLA-8","meanLA-16", "meanCO-2","meanCO-4"]].values

    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state=self._data.loc[0,["meanLA","meanCO","meanLA-4","meanLA-8","meanLA-16", "meanCO-2","meanCO-4"]].values
    self._i=3
    self._IA=0
    self._VA=0
    self._VC=0
    self._na=0
    self._invested=0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.float32))
   

  def _step(self, action):
    if self._i>=self._Lon:
        self._episode_ended = True



#ACCION 2
    if action == 2  and self._IA<=3*self._C:
      self._VC=self._VC+self._data.loc[self._i+1,"Open"]*int(self._C/self._data.loc[self._i+1,"Open"])
      self._na=self._na+int(self._C/self._data.loc[self._i+1,"Open"])
      self._VA=self._VC      
      self._IA+=self._VC
      #self._reward=0.0
      self._invested=1

    elif action == 2 :         
      self._na=int(self._C/self._data.loc[self._i+1,"Open"])
      self._VA=self._data.loc[self._i+1,"Open"]*self._na
      self._VC=self._data.loc[self._i+1,"Open"]*self._na
      self._IA+=self._VC
      #self._reward=0.0   
      self._invested=1
# ACCION 1
    #elif action == 1  and self._VC>0:
      #self._reward=self._data.loc[self._i+1,"Open"]*self._na-self._VC
      #self._IA-=self._VC
      #self._IA=self._IA
      #self._VC=0
      #self._na=0
      #self._VA=0
      #self._invested=0
    elif action == 1:
      self._IA=self._IA
      self._VC=self._VC 
      self._na=self._na
      self._VA=self._VA 
      #self._reward=0        
      self._invested=0
 
 # ACCION 0
    elif action==0:          
      self._VC=self._VC
      self._IA=self._IA
      self._na=self._na
      self._VA=self._na*self._data.loc[self._i,"Close"] 
      #self._reward=0 
        
    else:
          
      raise ValueError('`action` should be 0, 1, or 2')
    # compute reward
    if self._invested:
      self._reward=(self._data.loc[self._i,"LogReturn"])*self._na
    else:
      self._reward = 0
    # baseline
    #self._total_buy_and_hold += self._df.loc[self._current_idx,['LogReturn']].values
    if self._episode_ended:
      self._IA=0
      self._VC=0
      self._na=0
      self._i=0
      self._reward = self._reward
      self._invested=0
      self._state=self._data.loc[0,["meanLA","meanCO","meanLA-4","meanLA-8","meanLA-16", "meanCO-2","meanCO-4"]].values
      return ts.termination(np.array([self._state], dtype=np.float32), self._reward)
    
    else:
      self._i=self._i+1
      self._IA=self._IA
      self._VC=self._VC
      self._na=self._na
      self._VA=self._na*self._data.loc[self._i,"Close"]
      self._reward=self._reward
      self._invested=self._invested
      self._state = self._data.loc[self._i,["meanLA","meanCO","meanLA-4","meanLA-8","meanLA-16", "meanCO-2","meanCO-4"]].values



      return ts.transition(np.array([self._state], dtype=np.float32), self._reward)


In [19]:
class EnEnv(py_environment.PyEnvironment):
  def __init__(self, df):
    self._df = df
    self._n = len(df)-1
    self._current_idx = 0
    self._total_buy_and_hold = 0
    self._invested = 0
    
    #self._states = self._df[feats].to_numpy()
    self._reward = self._df['LogReturn']#.to_numpy()
    
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=2, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,5), dtype=np.float32, minimum=0, maximum=1,  name='observation')
    self._state=self._df.loc[0,["meanLA","meanCO","meanCO-8","meanCO-16","meanCO-24"]].values

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec 

  def _reset(self):
    self._current_idx = 0
    self._total_buy_and_hold = 0
    self._invested = 0
    self._reward=0
    self._state=self._df.loc[0,["meanLA","meanCO","meanCO-8","meanCO-16","meanCO-24"]].values
    return ts.restart(np.array([self._state], dtype=np.float32))

  def _step(self, action):
    # need to return (next_state, reward, done)
    
    if self._current_idx >=self._n:
      self._current_idx = 0
      self._total_buy_and_hold = 0
      self._invested = 0
      self._reward=0
      self._state=self._df.loc[0,["meanLA","meanCO","meanCO-8","meanCO-16","meanCO-24"]].values
      return ts.termination(np.array([self._state], dtype=np.float32), self._reward)
    
    if self._current_idx >= self._n:
      raise Exception("Episode already done")

    if action == 0: # BUY
      self._invested = 1
    elif action == 1: # SELL
      self._invested = 0
    elif action == 2: # DO NOTHING
      self._invested = self._invested    
    # compute reward
    if self._invested:
      self._reward = self._df.loc[self._current_idx,'LogReturn']
    else:
      self._reward = 0
    # baseline
    self._total_buy_and_hold += self._df.loc[self._current_idx,['LogReturn']].values

    # state transition
    
    self._state = self._df.loc[self._current_idx,["meanLA","meanCO","meanCO-8","meanCO-16","meanCO-24"]].values
    self._current_idx = self._current_idx+1
    return ts.transition(np.array([self._state], dtype=np.float32), self._reward)



## Environment

In Reinforcement Learning (RL), an environment represents the task or problem to be solved. Standard environments can be created in TF-Agents using `tf_agents.environments` suites. TF-Agents has suites for loading environments from sources such as the OpenAI Gym, Atari, and DM Control.

Load the CartPole environment from the OpenAI Gym suite. 

In [20]:
env = EnEnv(data)
print('action_spec:', env.action_spec())
print("")

print(env.time_step_spec())
print("")
print('time_step_spec.observation:', env.time_step_spec().observation)
print('time_step_spec.step_type:', env.time_step_spec().step_type)
print('time_step_spec.discount:', env.time_step_spec().discount)
print('time_step_spec.reward:', env.time_step_spec().reward)

action_spec: BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=2)

TimeStep(step_type=ArraySpec(shape=(), dtype=dtype('int32'), name='step_type'), reward=ArraySpec(shape=(), dtype=dtype('float32'), name='reward'), discount=BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0), observation=BoundedArraySpec(shape=(1, 5), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0))

time_step_spec.observation: BoundedArraySpec(shape=(1, 5), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)
time_step_spec.step_type: ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')
time_step_spec.discount: BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0)
time_step_spec.reward: ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


The `environment.step` method takes an `action` in the environment and returns a `TimeStep` tuple containing the next observation of the environment and the reward for the action.

The `time_step_spec()` method returns the specification for the `TimeStep` tuple. Its `observation` attribute shows the shape of observations, the data types, and the ranges of allowed values. The `reward` attribute shows the same details for the reward.


In [21]:
print('Observation Spec:')
print(env.time_step_spec().observation)

Observation Spec:
BoundedArraySpec(shape=(1, 5), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)


In [22]:
print('Reward Spec:')
print(env.time_step_spec().reward)

Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')


The `action_spec()` method returns the shape, data types, and allowed values of valid actions.

In [23]:
print('Action Spec:')
print(env.action_spec())

Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=2)


In [24]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Time step:
TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([[0.663225  , 0.7897817 , 0.4758948 , 0.75510406, 0.7461012 ]],
      dtype=float32))
Next time step:
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([[0.663225  , 0.7897817 , 0.4758948 , 0.75510406, 0.7461012 ]],
      dtype=float32))


Usually two environments are instantiated: one for training and one for evaluation. 

In [25]:
train_py_env = EnEnv(data)
eval_py_env = EnEnv(data)

The Cartpole environment, like most environments, is written in pure Python. This is converted to TensorFlow using the `TFPyEnvironment` wrapper.

The original environment's API uses Numpy arrays. The `TFPyEnvironment` converts these to `Tensors` to make it compatible with Tensorflow agents and policies.


In [26]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [27]:
tf.random.set_seed(200)

## Agent

The algorithm used to solve an RL problem is represented by an `Agent`. TF-Agents provides standard implementations of a variety of `Agents`, including:

-   [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (used in this tutorial)
-   [REINFORCE](http://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
-   [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
-   [TD3](https://arxiv.org/pdf/1802.09477.pdf)
-   [PPO](https://arxiv.org/abs/1707.06347)
-   [SAC](https://arxiv.org/abs/1801.01290).

The DQN agent can be used in any environment which has a discrete action space.

At the heart of a DQN Agent is a `QNetwork`, a neural network model that can learn to predict `QValues` (expected returns) for all actions, given an observation from the environment.

Use `tf_agents.networks.q_network` to create a `QNetwork`, passing in the `observation_spec`, `action_spec`, and a tuple describing the number and size of the model's hidden layers.


In [28]:
fc_layer_params = (400,300)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

Now use `tf_agents.agents.dqn.dqn_agent` to instantiate a `DqnAgent`. In addition to the `time_step_spec`, `action_spec` and the QNetwork, the agent constructor also requires an optimizer (in this case, `AdamOptimizer`), a loss function, and an integer step counter.

In [29]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

tf_agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

tf_agent.initialize()

## Policies

A policy defines the way an agent acts in an environment. Typically, the goal of reinforcement learning is to train the underlying model until the policy produces the desired outcome.

In this tutorial:

-   The desired outcome is keeping the pole balanced upright over the cart.
-   The policy returns an action (left or right) for each `time_step` observation.

Agents contain two policies: 

-   `agent.policy` — The main policy that is used for evaluation and deployment.
-   `agent.collect_policy` — A second policy that is used for data collection.


In [30]:
eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

Policies can be created independently of agents. For example, use `tf_agents.policies.random_tf_policy` to create a policy which will randomly select an action for each `time_step`.

In [31]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

To get an action from a policy, call the `policy.action(time_step)` method. The `time_step` contains the observation from the environment. This method returns a `PolicyStep`, which is a named tuple with three components:

-   `action` — the action to be taken (in this case, `0` or `1`)
-   `state` — used for stateful (that is, RNN-based) policies
-   `info` — auxiliary data, such as log probabilities of actions

In [32]:
time_step = env.reset()

In [33]:
#random_policy.action(time_step)

## Metrics and Evaluation

The most common metric used to evaluate a policy is the average return. The return is the sum of rewards obtained while running a policy in an environment for an episode. Several episodes are run, creating an average return.

The following function computes the average return of a policy, given the policy, environment, and a number of episodes.


In [34]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

Running this computation on the `random_policy` shows a baseline performance in the environment.

In [35]:
#compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Replay Buffer

The replay buffer keeps track of data collected from the environment. This tutorial uses `tf_agents.replay_buffers.tf_uniform_replay_buffer.TFUniformReplayBuffer`, as it is the most common. 

The constructor requires the specs for the data it will be collecting. This is available from the agent using the `collect_data_spec` method. The batch size and maximum buffer length are also required.


In [36]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=tf_agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

For most agents, `collect_data_spec` is a named tuple called `Trajectory`, containing the specs for observations, actions, rewards, and other items.

In [37]:
tf_agent.collect_data_spec

Trajectory(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), observation=BoundedTensorSpec(shape=(1, 5), dtype=tf.float32, name='observation', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), action=BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(2, dtype=int32)), policy_info=(), next_step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)))

In [38]:
tf_agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

## Data Collection

Now execute the random policy in the environment for a few steps, recording the data in the replay buffer.

In [39]:
#@test {"skip": true}
def collect_step(environment, policy, buffer):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
  for _ in range(steps):
    collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

# This loop is so common in RL, that we provide standard implementations. 
# For more details see the drivers module.
# https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers

The replay buffer is now a collection of Trajectories.

In [40]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

The agent needs access to the replay buffer. This is provided by creating an iterable `tf.data.Dataset` pipeline which will feed data to the agent.

Each row of the replay buffer only stores a single observation step. But since the DQN Agent needs both the current and next observation to compute the loss, the dataset pipeline will sample two adjacent rows for each item in the batch (`num_steps=2`).

This dataset is also optimized by running parallel calls and prefetching data.

In [41]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)


dataset

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


<PrefetchDataset shapes: (Trajectory(step_type=(64, 2), observation=(64, 2, 1, 5), action=(64, 2), policy_info=(), next_step_type=(64, 2), reward=(64, 2), discount=(64, 2)), BufferInfo(ids=(64, 2), probabilities=(64,))), types: (Trajectory(step_type=tf.int32, observation=tf.float32, action=tf.int32, policy_info=(), next_step_type=tf.int32, reward=tf.float32, discount=tf.float32), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [42]:
iterator = iter(dataset)

print(iterator)


In [43]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy

In [46]:
policy_dir_best = os.path.join("/content/drive/MyDrive/policy_ibex", 'policy')

tf_policy_saver = policy_saver.PolicySaver(tf_agent.policy)

## Training the agent

Two things must happen during the training loop:

-   collect data from the environment
-   use that data to train the agent's neural network(s)

This example also periodicially evaluates the policy and prints the current score.

The following will take ~5 minutes to run.

In [ ]:

try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]
avg_return_b=0
for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, tf_agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = tf_agent.train(experience).loss

  step = tf_agent.train_step_counter.numpy()

  #if step % log_interval == 0:
    #print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return_i = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
    if avg_return_i>=avg_return_b:
      avg_return_b=avg_return_i
      tf_policy_saver.save(policy_dir_best)
      print('step = {0}: Average Return Best = {1}'.format(step, avg_return_b))
      returns.append(avg_return_b)
    else:
      print('step = {0}: Average Return = {1}'.format(step, avg_return_i))
      returns.append(avg_return_i)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


INFO:tensorflow:Assets written to: /content/drive/MyDrive/policy_ibex/policy/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/policy_ibex/policy/assets


step = 1000: Average Return Best = 0.7617636322975159
step = 2000: Average Return = 0.2876186966896057
step = 3000: Average Return = 0.6011011004447937
step = 4000: Average Return = 0.6428426504135132


## Visualization


### Plots

Use `matplotlib.pyplot` to chart how the policy improved during training.

One iteration of `Cartpole-v0` consists of 200 time steps. The environment gives a reward of `+1` for each step the pole stays up, so the maximum return for one episode is 200. The charts shows the return increasing towards that maximum each time it is evaluated during training. (It may be a little unstable and not increase monotonically each time.)

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=2)

## Aplicamos la policy a los diferentes tickers

In [ ]:
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy

In [ ]:
best_policy = tf.compat.v2.saved_model.load(policy_dir_best)

In [ ]:

def run_one_episodes(policy, eval_env,ticker):
  action_s=[]

  time_step = eval_env_accion.reset()
  ret=0
  while not time_step.is_last():
    action_step = policy.action(time_step)
    time_step = eval_env_accion.step(action_step.action)

    ret+=time_step.reward
    rent=ret.numpy()
    accion= action_step[0].numpy()
    action_s.append(accion[0])



  print(ticker,rent,len(action_s),action_s,"Recomendacion: ", accion)
  return rent[0],accion,ticker

In [ ]:
start="2020-10-10"
end="2021-5-30"
Return=0
compras=[]
ventas=[]
mantener=[]
for ticker in ibex:
  #action_s=[]

  data_ticker=Betas(ticker,start=start,end=end).betas()
  data_ticker.reset_index(inplace=True)
  data_ticker["ticker"]=ticker
  env_accion=EnEnv(data_ticker) #este es el objeto entorno con las propiedades y metodos (entre ellos el metodo step)
  #utils.validate_py_environment(env_accion, episodes=5)
  eval_env_accion = tf_py_environment.TFPyEnvironment(env_accion)

  #aplicamos la mejor policia  
  rentabilidad,accion,ticker=run_one_episodes(best_policy, eval_env_accion,ticker)
  Return=Return+rentabilidad
  if accion==0:
    compras.append(ticker)
  elif accion==1:
    ventas.append(ticker)
  elif accion==2:
    mantener.append(ticker)

  eval_env_accion.reset()
print("")
print("Rentablidad del periodo y mercado",Return)
print("")
print("Valores a comprar: ",compras)
print("Valores a vender: ",ventas)
print("Valores a mantener: ",mantener)

In [ ]:
start="2020-10-16"
end="2021-5-30"
Return=0
compras=[]
ventas=[]
mantener=[]
for ticker in ibex:
  #action_s=[]

  data_ticker=Betas(ticker,start=start,end=end).betas()
  data_ticker.reset_index(inplace=True)
  data_ticker["ticker"]=ticker
  env_accion=EnEnv(data_ticker) #este es el objeto entorno con las propiedades y metodos (entre ellos el metodo step)
  #utils.validate_py_environment(env_accion, episodes=5)
  eval_env_accion = tf_py_environment.TFPyEnvironment(env_accion)

  #aplicamos la mejor policia  
  rentabilidad,accion,ticker=run_one_episodes(tf_agent.policy, eval_env_accion,ticker)
  Return=Return+rentabilidad
  if accion==0:
    compras.append(ticker)
  elif accion==1:
    ventas.append(ticker)
  elif accion==2:
    mantener.append(ticker)

  eval_env_accion.reset()
print("")
print("Rentablidad del periodo y mercado",Return)
print("")
print("Valores a comprar: ",compras)
print("Valores a vender: ",ventas)
print("Valores a mantener: ",mantener)

In [ ]:
start="2020-10-16"
end="2021-5-30"
Return=0
for ticker in ibex:
  action_s=[]
  data_ticker=Betas(ticker,start=start,end=end).betas()
  data_ticker.reset_index(inplace=True)
  data_ticker["ticker"]=ticker
  rentabilidad=data_ticker["LogReturn"].sum()
  Return=Return+rentabilidad
  eval_env_accion.reset()
print("")
print("Rentabilidad buy and hold",Return)

In [ ]:
policy_dir = os.path.join("tutorials/tf2_course/Archivo/policy", 'policy')

tf_policy_saver = policy_saver.PolicySaver(tf_agent.policy)

In [ ]:
tf_policy_saver.save(policy_dir)

In [ ]:
saved_policy = tf.compat.v2.saved_model.load(policy_dir)

In [ ]:
saved_policy.action

In [ ]:
time_step = eval_env_accion.reset()
action_step = saved_policy.action(time_step)
time_step = eval_env_accion.step(action_step.action)
accion=action_step[0][0].numpy()
reward=time_step[1][0].numpy()
accion, reward, time_step

In [ ]:
eval_env

In [ ]:
def run_one_episod(policy, eval_env,ticker, data_ticker,L):
  #eval_env_accion = tf_py_environment.TFPyEnvironment(env_accion)
  eval_env_accion = tf_py_environment.TFPyEnvironment(eval_env)
  time_step = eval_env_accion.reset()
  obs=eval_env_accion.reset()[3][0][0].numpy()
  ret=eval_env_accion.reset()[1][0].numpy()
  
  operaciones=pd.DataFrame() 
  contador=1
  data_ticker.loc[contador-1,"invertido"]=False
  I=5000


  while contador <=L-1: 
  #while not time_step.is_last():
    
    #action= policy(obs)
    action_step = policy.action(time_step)
    #time_step = eval_env_accion.step(action_step.action)
    time_step = eval_env_accion.step(action_step.action)   

    time_step_type = time_step[0][0].numpy()
    obs=time_step[3][0][0].numpy()
    ret=time_step[1][0].numpy()
    accion=action_step[0][0].numpy()
    LogReturn=data_ticker.loc[contador,"LogReturn"]
    
    if accion==0:
      if data_ticker.loc[contador-1,"invertido"]==False:
        ret=float(ret)
        data_ticker.loc[contador,"accion"]=accion
        data_ticker.loc[contador,"invertido"]=True           
        data_ticker.loc[contador,"inversion"]=I
        data_ticker.loc[contador,"ganancia"]=0

      else:
        ret=float(ret)
        data_ticker.loc[contador,"accion"]=accion
        data_ticker.loc[contador,"invertido"]=True      
        data_ticker.loc[contador,"inversion"]=0
        data_ticker.loc[contador,"ganancia"]=LogReturn
    if accion==1:
      if data_ticker.loc[contador-1,"invertido"]==True:
        ret=float(ret)
        data_ticker.loc[contador,"accion"]=accion
        data_ticker.loc[contador,"invertido"]=False      
        data_ticker.loc[contador,"inversion"]=-I
        data_ticker.loc[contador,"ganancia"]=LogReturn
      else:
        ret=float(ret)
        data_ticker.loc[contador,"accion"]=accion
        data_ticker.loc[contador,"invertido"]=False      
        data_ticker.loc[contador,"inversion"]=0
        data_ticker.loc[contador,"ganancia"]=0     
    elif accion==2:
      if  data_ticker.loc[contador,"invertido"]==True:
        ret=float(ret)
        data_ticker.loc[contador,"accion"]=accion
        data_ticker.loc[contador,"invertido"]=True      
        data_ticker.loc[contador,"inversion"]=0
        data_ticker.loc[contador,"ganancia"]=LogReturn
      else:
        ret=float(ret)
        data_ticker.loc[contador,"accion"]=accion
        data_ticker.loc[contador,"invertido"]=False      
        data_ticker.loc[contador,"inversion"]=0
        data_ticker.loc[contador,"ganancia"]=0

    ticker=data_ticker.loc[contador,"ticker"]
    fecha=data_ticker.loc[contador,"Date"]
    cotizacion=data_ticker.loc[contador,"Close"]
    accion=data_ticker.loc[contador,"accion"]
    invertido=data_ticker.loc[contador,"invertido"]

    ganancia=data_ticker.loc[contador,"ganancia"]
    inversion=data_ticker.loc[contador,"inversion"]
    meanLA=data_ticker.loc[contador,"meanLA"]
    meanCO=data_ticker.loc[contador,"meanCO"]
    #Posteriori2=data_ticker.loc[contador,"Posteriori2"]
    #Posteriori3=data_ticker.loc[contador,"Posteriori3"]        
    LogReturn=data_ticker.loc[contador,"LogReturn"]
    

    df=pd.DataFrame([[ticker,fecha, cotizacion, accion, invertido, ganancia, inversion, meanLA, meanCO,LogReturn]],columns=["ticker","fecha","cotizacion","accion","invertido","ganancia","inversion","meanLA","meanCO","LogReturn"])

    operaciones=operaciones.append(df,ignore_index=True)

  
    contador+=1
  return operaciones

In [ ]:
start="2020-10-01"
end="2021-12-31"

Operaciones=pd.DataFrame()
for ticker in ibex:

  data_ticker1=Betas(ticker,start=start,end=end).betas()
  data_ticker=data_ticker1.iloc[-250:].copy()
  data_ticker.reset_index(inplace=True)
  
  data_ticker["ticker"]=ticker
  data_ticker['Date_'] =  pd.to_datetime(data_ticker.Date)
  L=len(data_ticker)
  env=EnEnv(data_ticker)

  operaciones7=run_one_episod(best_policy, env,ticker,data_ticker,L)


  Operaciones=Operaciones.append([operaciones7],ignore_index=True)

Operaciones.head(15)

In [ ]:
Operaciones.head(25)

In [ ]:
Operaciones.sort_values("fecha",inplace=True)
Operaciones[Operaciones["invertido"]==True]


In [ ]:
GananciaTotal=Operaciones["ganancia"].sum()
InversionFinal=Operaciones["inversion"].max()
print(GananciaTotal)
print(InversionFinal)

In [ ]:

Acciones=Operaciones[(Operaciones["accion"]==0)].count()
Acciones

In [ ]:
Operaciones.describe()

In [ ]:
Operaciones.sort_values("fecha",inplace=True)


In [ ]:
Operaciones['inversion_acum'] =Operaciones['inversion'].cumsum()
Operaciones['ganancia_acum'] =Operaciones['ganancia'].cumsum()
Operaciones["resultado"]=np.where((Operaciones['ganancia']>=0) , 1, 0)
Operaciones["resultado +"]=np.where((Operaciones['ganancia']>0)  , 1, 0)
Operaciones["resultado -"]=np.where((Operaciones['ganancia']<0)  , 1, 0)
Operaciones=Operaciones.copy()
Operaciones

In [ ]:
Operaciones["inversion_acum"].max(),Operaciones.loc[Operaciones.index[-1],"ganancia_acum"]

In [ ]:
Periodo=Operaciones.loc[Operaciones.index[-1],"fecha"]-Operaciones.loc[Operaciones.index[0],"fecha"]

In [ ]:
Periodo=Operaciones.loc[Operaciones.index[-1],"fecha"]-Operaciones.loc[Operaciones.index[0],"fecha"]
Inversion_maxima=Operaciones["inversion_acum"].max()
Ganancia_acumulada=Operaciones.loc[Operaciones.index[-1],"ganancia_acum"]
Rentabilidad_periodo= Ganancia_acumulada*100/Inversion_maxima
Ratio_operaciones_positivas=Operaciones["resultado +"].sum()*100/(Operaciones["resultado +"].sum()+Operaciones["resultado -"].sum())
Compras=len(Operaciones[(Operaciones["inversion"]>0) & (Operaciones["accion"]==0)])
Beneficio_x_compra= Ganancia_acumulada*5000/Compras
print("Periodo analizado:",Periodo,Operaciones.loc[Operaciones.index[0],"fecha"],Operaciones.loc[Operaciones.index[-1],"fecha"] )
print("Total compras realizadas             ",Compras)
print("Inversion Maxima:                 ","{:.0f}".format(Inversion_maxima ))
print("Rentabilidad Periodo (%):          ","{:.3f}".format(Ganancia_acumulada))
print("Ratio dias con rent positivas(%):  ","{:.2f}".format(Ratio_operaciones_positivas))
print("Beneficio por compra:             ", "{:.2f}".format(Beneficio_x_compra))
print("Beneficio total del periodo:       ","{:.0f}".format(Beneficio_x_compra*Compras ))

In [ ]:
# plot it
#fig, ax = plt.subplots(1,1)
fig, ax = plt.subplots(figsize=(12, 12))
#ax.hist(Operaciones["meanLA"], bins=10, color='lightblue',cumulative=False)

#ax.hist(Operaciones["meanCO"], bins=10, color='darkblue',cumulative=False)
ax.hist(Operaciones["ganancia"], bins=100, color='darkblue',cumulative=False)
#ax.xaxis.set_major_locator(mdates.YearLocator())
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y'))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
plt.figure(figsize=(15,15))

matplotlib.pyplot.grid(b=True)
#plt.hist(data["mean5"],20)
#plt.hist(data["mean34"],20)
plt.scatter(Operaciones["meanLA"],Operaciones["meanCO"],c=Operaciones["resultado +"])
#plt.scatter(Operaciones["meanLA"],Operaciones["meanCO"], c=Operaciones["ganancia"],cmap=ListedColormap(colors))
plt.xlabel(r'Posteriori', fontsize=15)
plt.ylabel(r'meanCO', fontsize=15)
plt.title('Probabilidades')
plt.axis([0, 1, 0, 1])


x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100,axis=0)
#Generamos una grafica lineal para una recta en X
plt.plot(x, (4/35)*x, label='compra 1')
plt.plot(x, (40/11)*x-5.5/11.0, label='compra 2')
plt.plot(x, (40/11)*x-13/11.0, label='venta 1')
plt.plot(x, (4/35)*x+3/35, label='venta 1')
plt.plot(x, 0.40/x, label='venta 3')
plt.plot(x, 1-0.025/(0.9*x**2), label='compra 3')

#Agregamos las etiquetas y añadimos una leyenda.
#plt.xlabel('Eje X')
#plt.ylabel('Eje Y')
#plt.title("Simple Plot")
#plt.legend()
plt.savefig('grafica_lineal.png')
plt.show()




plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
plt.plot(Operaciones["fecha"],Operaciones["inversion_acum"])
plt.plot(Operaciones["fecha"],Operaciones["ganancia_acum"]*5000)
plt.show()

In [ ]:
start="2010-01-01"
end="2020-12-31"
dataset=pd.DataFrame()
for ticker in ibex:
  data_ticker1=Betas(ticker,start=start,end=end).betas()
  data_ticker=data_ticker1.iloc[-25:].copy()
  dataset=dataset.append(data_ticker,ignore_index=True)

#mercados=[igbm,ibex,cac,dax,mib,dow,nasdaq,s_and_p_100]
mercados=[ibex,cac,dax,mib,dow]
for mercado in mercados:
  vender=[]
  previo=[]
  mantener_posicion=[]
  comprar=[]
  for ticker in mercado:
    datas=Betas(ticker,start=start,end=end).betas()
    F=datas.index[-1]
    ultimo=datas.loc[F, ["meanLA"]]    
    ult=np.asarray(ultimo)
    ultimop=datas.loc[F, ["meanCO"]]
    ultp=np.asarray(ultimop)

    if ult==2:
      comprar.append(ticker)

    elif ultp==2:
      previo.append(ticker)

    elif ult==0:
      mantener_posicion.append(ticker)
    elif ult==1:
      vender.append(ticker)
    else:
      mantener_posicion.append(ticker)

In [ ]:
start="2020-01-01"
end="2020-12-31"
dataset=pd.DataFrame()
for ticker in ibex:
  data_ticker1=Betas(ticker,start=start,end=end).betas()
  data_ticker=data_ticker1.iloc[-1:].copy()
  data_ticker["Ticker"]=ticker
  obs=eval_env.reset()[3][0][0].numpy()
  ret=eval_env.reset()[1][0].numpy()
  action,EC= best_policy(obs,0)  
  print("Ticker",data_ticker.loc[-1:,"Ticker"],"Accion",action)
  